In [190]:
class Intcode:
    
    def __init__(self, code, loop=False, phase=None):
        
        self.intcode = [int(x) for x in code.strip().split(',')]
        self.intcode.extend([0]*10000)
        
        self.i = 0
        
        self.input = []
        self.output = []
        
        self.loop = loop
        self.halt = None
        
        self.base = 0
        
        self.verbose = False
        
        if phase is not None:
            self.input.append(phase)
        
        
    def __call__(self):
        self.compute()

    def set_input(self, input):
        self.input = input
        
        
    def get_output(self):
        return self.output

    
    def set_loop(self):
        self.loop = True
    
    
    def set(self, i, value, mode=0):
    
        if mode == 0:
            self.intcode[i] = value
            
        elif mode == 1:
            self.intcode[i] = value
        
        elif mode == 2:
            self.intcode[i + self.base] = value
    
        else:
            raise ValueError
            
    
    
    def get(self, i=0, mode=0, verbose=False):
            
        if verbose:
            print("@", i, mode, self.base)
            
        if mode == 0:
            return self.intcode[i]
        
        elif mode == 1:
            return i
        
        elif mode == 2:
            #print(self.base)
            return self.intcode[i + self.base] 
        
        else:
            raise ValueError

    def print_intcode(self):
        print(self.intcode)
            
    def compute(self):
        
        
        while self.i < len(self.intcode):
            
            opcode = int(str(self.intcode[self.i])[-2:])
            modes = str(self.intcode[self.i])[:-2].zfill(3)
            
            if self.verbose:
                print(opcode, modes)
            #print(self.intcode[1000])
            #print(opcode, modes, self.i, len(self.intcode))
            #self.print_intcode()
            #print(self.input, self.output)
            #print()
            
            nparam = getattr(self, "compute{}".format(opcode))(modes=modes)

            if opcode == 99:
                return
            
            
            self.i += nparam +1

            if opcode == 4 and self.loop:
                return
            
            
    def compute1(self, nparam=3, modes="000"):
        
        self.set(self.intcode[self.i+3],
                 self.get(self.intcode[self.i+1], mode=int(modes[-1])) +
                 self.get(self.intcode[self.i+2], mode=int(modes[-2])),
                 mode=int(modes[-3]))
            
        return nparam
        
    def compute2(self, nparam=3, modes="000"):
                 
        self.set(self.intcode[self.i+3],
                 self.get(self.intcode[self.i+1], mode=int(modes[-1])) *
                 self.get(self.intcode[self.i+2], mode=int(modes[-2])),
                 mode=int(modes[-3]))
        
        return nparam
    
    def compute3(self, nparam=1, modes="1", keyboard=False):
        """input"""
        if keyboard:
            n = int(input("Enter integer:"))
        else:
            n = self.input.pop(0)
    
        self.set(self.intcode[self.i+1], n,  mode=int(modes[-1]))
        return nparam
    
    def compute4(self, nparam=1, modes="0"):
        """output"""
        output = self.get(self.intcode[self.i+1], mode=int(modes[-1]))
        print(">", output)
        self.output.append(output)
        return nparam
    
    def compute5(self, nparam=2, modes="00"):
        """jump-if-true"""
        
        if self.get(self.intcode[self.i+1], mode=int(modes[-1])):
            self.i = self.get(self.intcode[self.i+2], mode=int(modes[-2]))
            return -1       
        
        return nparam
    
    def compute6(self, nparam=2, modes="00"):
        """jump-if-false"""
        
        if not self.get(self.intcode[self.i+1], mode=int(modes[-1])):
            self.i = self.get(self.intcode[self.i+2], mode=int(modes[-2]))
            return -1       
        
        return nparam
    
    def compute7(self, nparam=3, modes="000"):
        """less than"""

        if self.get(self.intcode[self.i+1], mode=int(modes[-1])) < self.get(self.intcode[self.i+2], mode=int(modes[-2])):
            self.set(self.intcode[self.i+3], 1, mode=int(modes[-3]))
        else:
            self.set(self.intcode[self.i+3], 0, mode=int(modes[-3]))
        return nparam
    
    def compute8(self, nparam=3, modes="000"):
        """equals"""
        if self.get(self.intcode[self.i+1], mode=int(modes[-1])) == self.get(self.intcode[self.i+2], mode=int(modes[-2])):
            self.set(self.intcode[self.i+3], 1, mode=int(modes[-3]))

        else:
            self.set(self.intcode[self.i+3], 0, mode=int(modes[-3]))
            
        return nparam
    
    def compute9(self, nparam=1, modes="0"):
        """adjust relative base"""
        self.base += self.get(self.intcode[self.i+1], mode=int(modes[-1]))
        return nparam
    
    def compute99(self, nparam=1, modes="0"):
        self.halt = True
        return 
      

In [195]:
code = "109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99"
Intcode(code)()

> 109
> 1
> 204
> -1
> 1001
> 100
> 1
> 100
> 1008
> 100
> 16
> 101
> 1006
> 101
> 0
> 99


In [193]:
code = "1102,34915192,34915192,7,4,7,99,0"
Intcode(code)()

> 1219070632396864


In [194]:
code = "104,1125899906842624,99"
Intcode(code)()

> 1125899906842624


In [192]:
with open("input.txt") as f:
    code = f.read()
print(code)
    
intcode = Intcode(code)
intcode.set_input([1])
intcode()


1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1102,1,3,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,902,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1102,1,37,1007,1102,24,1,1006,1102,26,1,1012,1101,528,0,1023,1102,256,1,1027,1102,466,1,1029,1102,1,629,1024,1101,0,620,1025,1101,0,0,1020,1102,1,30,1004,1101,39,0,1003,1102,36,1,1005,1102,531,1,1022,1102,32,1,1019,1101,0,27,1000,1101,0,28,1016,1101,1,0,1021,1101,23,0,1013,1102,1,25,1015,1102,1,21,1008,1102,1,22,1018,1102,1,34,1014,1102,475,1,1028,1101,33,0,1002,1101,0,35,1011,1102,1,20,1009,1102,38,1,1017,1101,259,0,1026,1101,31,0,1010,1101,0,29,1001,109,8,21102,40,1,10,1008,1018,40,63,1005,63,203,4,187,1105,1,207,1001,64,1,64,1002,64,2,64,109,7,21108,41,41,0,1005,1015,225,4,213,1106,0,229,1001,64,1,64,1002,64,2,64,109,1,1205,5,247,4,235,1001,64,1,64,1105,1,247,1002,64,2,64,109,20,2106,0,-9,1105,1,265,4,253,1001,64,1,64,1002,64,2,64,109,-38,1202,4,1,63,1

```--- Part Two ---```

In [197]:
with open("input.txt") as f:
    code = f.read()
print(code)
    
intcode = Intcode(code)
intcode.set_input([2])
intcode()

1102,34463338,34463338,63,1007,63,34463338,63,1005,63,53,1102,1,3,1000,109,988,209,12,9,1000,209,6,209,3,203,0,1008,1000,1,63,1005,63,65,1008,1000,2,63,1005,63,902,1008,1000,0,63,1005,63,58,4,25,104,0,99,4,0,104,0,99,4,17,104,0,99,0,0,1102,1,37,1007,1102,24,1,1006,1102,26,1,1012,1101,528,0,1023,1102,256,1,1027,1102,466,1,1029,1102,1,629,1024,1101,0,620,1025,1101,0,0,1020,1102,1,30,1004,1101,39,0,1003,1102,36,1,1005,1102,531,1,1022,1102,32,1,1019,1101,0,27,1000,1101,0,28,1016,1101,1,0,1021,1101,23,0,1013,1102,1,25,1015,1102,1,21,1008,1102,1,22,1018,1102,1,34,1014,1102,475,1,1028,1101,33,0,1002,1101,0,35,1011,1102,1,20,1009,1102,38,1,1017,1101,259,0,1026,1101,31,0,1010,1101,0,29,1001,109,8,21102,40,1,10,1008,1018,40,63,1005,63,203,4,187,1105,1,207,1001,64,1,64,1002,64,2,64,109,7,21108,41,41,0,1005,1015,225,4,213,1106,0,229,1001,64,1,64,1002,64,2,64,109,1,1205,5,247,4,235,1001,64,1,64,1105,1,247,1002,64,2,64,109,20,2106,0,-9,1105,1,265,4,253,1001,64,1,64,1002,64,2,64,109,-38,1202,4,1,63,1